In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Stratified k-fold cross-validation
from sklearn.model_selection import KFold, StratifiedKFold, cross_val_score

#RandomForest
from sklearn.ensemble import RandomForestClassifier as RFC

# GridSearch
from sklearn.model_selection import GridSearchCV

# 指標を計算するため
from sklearn.metrics import accuracy_score, auc, plot_roc_curve, confusion_matrix, roc_curve

# 標準化
from sklearn.preprocessing import StandardScaler

# seaborn
import seaborn as sns
sns.set()

###  下記内容を適宜変更してください。

## データはcsv形式 
## データ内のsubject列をlabelにします。

## InternalValidationとして使用するデータ
df=pd.read_csv('Internal_validation.csv')

## ExternalValidationとして使用するデータ
df_ex=pd.read_csv('External_validation.csv')

## Training dataにおける特徴量数 
num_feature = len(df.columns[1:])

## fold数
num_fold = 10

In [ ]:
## Internal Validation Cohort

#X yの設定
#Xの.dropで行or列を削除、axis=1で行を削除と指定。削除する行の指定は'subject'で行う。
X = df.drop('subject',axis=1) 

#yはdf内からsubjectのみを取り出す。
y = df.subject

# label設定
label = df.columns[1:]

In [ ]:
# モデル選択
model = RFC(random_state = 0, n_jobs = -1, oob_score=True)

# パラメーター設定
param_grid = {"n_estimators":[100,500,1000], 
              "max_features": [1, 2, 3, 4, 5, 7, 10],
              "max_depth": [3,5,7,10,15,None], 
              "min_samples_leaf":  [1, 2, 4],
              "min_samples_split": [2, 5, 10]
             } 

# Confusion matrix
tn = []
fp = []
fn = []
tp = []
tprs = []
aucs = []

accuracy_list = []
sensitivity_list = []
specificity_list = []

mean_accuracy = []
mean_sensitivity = []
mean_specificity = []

iteration_accuracy = []
iteration_sensitivity = []
iteration_specificity = []

# 標準化
stdsc = StandardScaler()

# 使用するtraining dataの特徴量数。
imp_list = np.empty([num_feature, ])

## fold数
num_fold = 10

In [ ]:
%%time
fig, ax = plt.subplots(figsize=(12,10))
mean_fpr = np.linspace(0, 1, 100)
    
# 層化k分割交差検証　n_splits=10のため 1/10 * 10回での検討
    
cv_cross_val = StratifiedKFold(n_splits=10, shuffle=True, random_state=1000)

cv_gs = StratifiedKFold(n_splits=10, shuffle=True, random_state=1000)
    
#  Stratified cross validation
for i, (train, test) in enumerate(cv_cross_val.split(X, y)):
    
    print(f"========= fold:{i + 1} =========")
            
    # train, testへ分割
    X_train, X_test = X.iloc[train], X.iloc[test]
    y_train, y_test = y.iloc[train], y.iloc[test]
            
    # training data, test dataをそれぞれ標準化
    X_train_std, X_test_std = pd.DataFrame(stdsc.fit_transform(X_train)), pd.DataFrame(stdsc.transform(X_test))
        
    # Grid search
    gs = GridSearchCV(estimator = model,
                      param_grid = param_grid,
                      scoring = 'accuracy',
                      cv = cv_gs,
                      return_train_score = True,
                      n_jobs = -1)
    
    gs.fit(X_train_std, y_train)
    
    print("best_estimator", gs.best_estimator_)
    
    clf = gs.best_estimator_
    
    clf.fit(X_train_std, y_train)
    
    y_pred = clf.predict(X_test_std)
    y_true = y_test

    confmat = confusion_matrix(y_true, y_pred)
    tn.append(confusion_matrix(y_true, y_pred)[0][0])
    fp.append(confusion_matrix(y_true, y_pred)[0][1])
    fn.append(confusion_matrix(y_true, y_pred)[1][0])
    tp.append(confusion_matrix(y_true, y_pred)[1][1])
    
    accuracy = ((np.array(tp)+np.array(tn))/(np.array(tp)+np.array(fp)+np.array(fn)+np.array(tn)))
    sensitivity = (np.array(tp)/(np.array(tp)+np.array(fn)))
    specificity = (np.array(tn)/(np.array(tn)+np.array(fp)))
    
    # ROC curve
    viz = plot_roc_curve(clf, X_test_std, y_test, name='ROC'.format(i), alpha=0, lw=0, ax=ax)
    interp_tpr = np.interp(mean_fpr, viz.fpr, viz.tpr)
    interp_tpr[0] = 0.0
    tprs.append(interp_tpr)
    aucs.append(viz.roc_auc)
    
    mean_tpr = np.mean(tprs, axis=0)
    mean_tpr[-1] = 1.0
    mean_auc = auc(mean_fpr, mean_tpr)
    std_auc = np.std(aucs)
    std_tpr = np.std(tprs, axis=0)
    
    # feature_importances_
    clf_imp = clf.feature_importances_
    print(f"importance fold({i + 1}) :", clf_imp)
    
    imp_list = imp_list + clf_imp
    print('importance list:', imp_list)
    
 # 全foldの指標をリスト化 → Iteration終了後に計算する
print('Accuracy:', accuracy)
print('Sensitivity:', sensitivity)
print('Specificity:', specificity)

accuracy_list.append(accuracy)
sensitivity_list.append(sensitivity)
specificity_list.append(specificity)
    
## 10-fold cross validationを1 iteration 試行した結果の各種指標。(=通常のCV)
    
#各foldごとの指標
mean_accuracy = np.mean(accuracy)
mean_sensitivity = np.mean(sensitivity)
mean_specificity = np.mean(specificity)
    
mean_accuracy_std = np.std(accuracy)
mean_sensitivity_std = np.std(sensitivity)
mean_specificity_std = np.std(specificity)

# ROC curve描画
ax.plot(mean_fpr, mean_tpr, color='green', label=r'Mean ROC (AUC = %0.2f $\pm$ %0.2f)' % (mean_auc, std_auc), lw=4, alpha=.8)
ax.plot([0, 1], [0, 1], linestyle='--', lw=4, color='r', label='Chance', alpha=.8)
tprs_upper = np.minimum(mean_tpr + std_tpr, 1)
tprs_lower = np.maximum(mean_tpr - std_tpr, 0)
ax.fill_between(mean_fpr, tprs_lower, tprs_upper, color='grey', alpha=0.2, label=r'$\pm$ 1 std. dev.')
ax.set(xlim=[-0.02, 1.02], ylim=[-0.02, 1.02]) 
ax.legend(loc="lower right")
plt.legend(bbox_to_anchor=(1.05, 0), loc='lower left', borderaxespad=0, fontsize=20)
plt.tick_params(labelsize=18)
plt.xlabel("")
plt.ylabel("")
plt.show()
fig.savefig('ROC_RF_InternalValidation.png', bbox_inches='tight')

# feature_importance
mean_imp_list = imp_list / num_fold
print("mean importance list", mean_imp_list)

In [ ]:
# 各種指標の出力
print ('mean_accuracy:', mean_accuracy)
print('mean_accuracy_std:', mean_accuracy_std)
print ('mean_sensitivity:', mean_sensitivity)
print('mean_sensitivity_std:', mean_sensitivity_std)
print ('mean_specificity:', mean_specificity)
print('mean_specificity_std:', mean_specificity_std)

In [ ]:
#  縦軸を特徴量、横軸をImportance_meanとしたDataframeを作成
df_importance = pd.DataFrame(zip(label, mean_imp_list), columns=["Features","Importance"])

# Importance_meanの順に並べ替え、上位10件のみを抽出する
df_importance = df_importance.sort_values("Importance", ascending=False).head(10)

# Figureの作成
plt.figure(figsize=(10,5))

sns.barplot(x="Importance", y="Features", data=df_importance, ci=95)
plt.title("Feature Importance")
plt.tight_layout()

In [ ]:
## External Validation Cohort

#X yの設定
#Xの.dropで行or列を削除、axis=1で行を削除と指定。削除する行の指定は'subject'で行う。
X_ex = df_ex.drop('subject',axis=1) 

#yはdf内からsubjectのみを取り出す。
y_ex = df_ex.subject

In [ ]:
## Training dataをInternal Validation data, Test dataをExternal Validation dataとしてmodelを一つ作成する。
# Training data全体を標準化
X_std, X_ex_std = pd.DataFrame(stdsc.fit_transform(X)), pd.DataFrame(stdsc.transform(X_ex))

# Confusion matrix
tn_ex = []
fp_ex = []
fn_ex = []
tp_ex = []
tprs_ex = []
aucs_ex = []
mean_fpr_ex = np.linspace(0, 1, 100)

## Grid search
# 処理はInternal Validationと同様だが、別のGridSearch CVを用意

cv_gs_ex = StratifiedKFold(n_splits=10, shuffle=True, random_state=1000)

gs_ex = GridSearchCV(estimator = model,
                     param_grid = param_grid,
                     scoring = 'accuracy',
                     cv = cv_gs_ex,
                     return_train_score = True,
                     n_jobs = -1)

gs_ex.fit(X_std, y)
        
print("best_estimator", gs_ex.best_estimator_)
            
clf_ex = gs_ex.best_estimator_
            
clf_ex.fit(X_std, y)
            
y_ex_pred = clf_ex.predict(X_ex_std)
y_ex_true = y_ex
        
confmat_ex = confusion_matrix(y_ex_true, y_ex_pred)
tn_ex.append(confusion_matrix(y_ex_true, y_ex_pred)[0][0])
fp_ex.append(confusion_matrix(y_ex_true, y_ex_pred)[0][1])
fn_ex.append(confusion_matrix(y_ex_true, y_ex_pred)[1][0])
tp_ex.append(confusion_matrix(y_ex_true, y_ex_pred)[1][1])
        
accuracy_ex = ((np.array(tp_ex)+np.array(tn_ex))/(np.array(tp_ex)+np.array(fp_ex)+np.array(fn_ex)+np.array(tn_ex)))
sensitivity_ex = (np.array(tp_ex)/(np.array(tp_ex)+np.array(fn_ex)))
specificity_ex = (np.array(tn_ex)/(np.array(tn_ex)+np.array(fp_ex)))

# ROC curve描画
fig, ax = plt.subplots(figsize=(12,10))
plot_roc_curve(clf_ex, X_ex_std, y_ex, color='green', name = 'ROC',  lw=4, alpha=.8, ax=ax)  
ax.plot([0, 1], [0, 1], linestyle='--', lw=4, color='r', label='Chance', alpha=.8)
ax.set(xlim=[-0.02, 1.02], ylim=[-0.02, 1.02])
plt.tick_params(labelsize=18)
plt.xlabel("")
plt.ylabel("")
ax.legend(loc="lower right")
plt.legend(bbox_to_anchor=(1.05, 0), loc='lower left', borderaxespad=0, fontsize=20)
plt.show()
fig.savefig('ROC_RF_ExternalValidation.png', bbox_inches='tight')
            
# 各試行における指標
print('Accuracy_ex:', accuracy_ex)
print('Sensitivity_ex:', sensitivity_ex)
print('Specificity_ex:', specificity_ex)